<a href="https://colab.research.google.com/github/saravanamuthu1/sentiment_analysis/blob/main/sentiment_analysis_sractch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
mylist=[]
testfile = open('train-v2.tsv', 'r')
for line in testfile:
  value=line.rstrip("\n").split("\t")
  mylist.append(line.rstrip("\n").split("\t"))


In [ ]:
train_set,test_set=mylist[0:70000],mylist[70000:80000]

get the probability

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
import re

In [ ]:
df=pd.DataFrame(mylist)

df.columns=["label","tweet"]

In [ ]:
df

In [ ]:
df1=pd.DataFrame(train_set)
df1.columns=["train_label","train_value"]

In [ ]:
df1

In [ ]:
df2=pd.DataFrame(test_set)
df2.columns=["test_label","test_value"]

In [ ]:
positive_frequency=len(df.loc[df.label == '1'])
negative_frequency=len(df.loc[df.label == "0"])
print(positive_frequency)
print(negative_frequency)
from collections import Counter

In [ ]:
positive_frequency=len(df1.loc[df1.train_label == '1'])
negative_frequency=len(df1.loc[df1.train_label == "0"])
print(positive_frequency)
print(negative_frequency)

In [ ]:
df['tweet'] = df.tweet.apply(lambda x:re.sub(r'[^\w\s]', '', x))
df['tweet'] = df.tweet.apply(lambda x:x.strip())
df['tweet'] = df.tweet.apply(lambda x:x.replace("USER", ""))

In [ ]:
df1['train_value'] = df1.train_value.apply(lambda x:re.sub(r'[^\w\s]', '', x))
df1['train_value'] = df1.train_value.apply(lambda x:x.strip())
df1['train_value'] = df1.train_value.apply(lambda x:x.replace("USER", ""))

In [ ]:
df2['test_value'] = df2.test_value.apply(lambda x:re.sub(r'[^\w\s]', '', x))
df2['test_value'] = df2.test_value.apply(lambda x:x.strip())
df2['test_value'] = df2.test_value.apply(lambda x:x.replace("@USER", ""))

In [ ]:
df['tweet'] =df.tweet.apply(lambda x:''.join(x))
df["tweet"]= df.tweet.apply(lambda x:" ".join(x.split()))
 

In [ ]:
df1['train_value'] =df1.train_value.apply(lambda x:''.join(x))
df1["train_value"]= df1.train_value.apply(lambda x:" ".join(x.split()))
 

In [ ]:
V_freq = Counter(" ".join(df['tweet'].values.tolist()).split(" "))
print(len(V_freq))

In [ ]:
V_freq_train=Counter(" ".join(df1['train_value'].values.tolist()).split(" "))

In [ ]:
freq_pos = Counter(" ".join(df.loc[df.label=='1']['tweet'].values.tolist()).split(" ")) 

freq_neg = Counter(" ".join(df.loc[df.label=='0']['tweet'].values.tolist()).split(" ")) 


In [ ]:
freq_pos_train = Counter(" ".join(df1.loc[df1.train_label=='1']['train_value'].values.tolist()).split(" ")) 

freq_neg_train = Counter(" ".join(df1.loc[df1.train_label=='0']['train_value'].values.tolist()).split(" ")) 

In [ ]:
print(len(freq_pos))
print(len(freq_neg))

In [ ]:
print(len(freq_pos_train))
print(len(freq_neg_train))

In [ ]:
total_porobaility_for_pos=((len(freq_pos)/len(V_freq)))
print(total_porobaility_for_pos)

In [ ]:
total_porobaility_for_neg=(len(freq_neg)/len(V_freq))
print(total_porobaility_for_neg)

In [ ]:
total_porobaility_for_pos_train=((len(freq_pos_train)/len(V_freq_train)))
print(total_porobaility_for_pos_train)
total_porobaility_for_neg_train=(len(freq_neg_train)/len(V_freq_train))
print(total_porobaility_for_neg_train)

In [ ]:
def pos_word_loglikelihood(w):
    if w in freq_pos:
      p_w_pos=float(freq_pos.get(w,0))/len(V_freq)
      return p_w_pos
    else:
      return 0
    

In [ ]:
def neg_word_loglikelihood(w):
  if w in freq_neg:
    p_w_neg =float(freq_neg.get(w,0))/len(V_freq)
    return p_w_neg
  return 0


In [ ]:
def pos_word_loglikelihood_train(w):
    if w in freq_pos_train:
      p_w_pos=float(freq_pos_train.get(w,0))/len(V_freq_train)
      return p_w_pos
    else:
      return 0

In [ ]:
def neg_word_loglikelihood_train(w):
  if w in freq_neg_train:
    p_w_neg =float(freq_neg_train.get(w,0))/len(V_freq_train)
    return p_w_neg
  return 0


In [ ]:
def total_score_label(mytxt):
  mytxt=mytxt.split()
  mytxt=list(set(mytxt))
  pos_score = total_porobaility_for_pos 
  neg_score = total_porobaility_for_neg
  for w in mytxt:
      pos_score= pos_score*pos_word_loglikelihood(w)
      neg_score= neg_score*neg_word_loglikelihood(w)
  max_value=max(pos_score,neg_score)
  if max_value == pos_score:
    return 1
  else:
    return 0



In [ ]:
df['total_prediction']=df.tweet.apply(lambda x:total_score_label(x))

df1['total_prediction_train']=df1.train_value.apply(lambda x:total_score_label(x))

df2['total_prediction_test']=df2.test_value.apply(lambda x:total_score_label(x))


In [ ]:
df['label'] = df.label.apply(lambda x:int(x))

In [ ]:
df1['train_label'] = df1.train_label.apply(lambda x:int(x))
accuracy1 = accuracy_score(df1['train_label'], df1['total_prediction_train'])

In [ ]:
df2['test_label'] = df2.test_label.apply(lambda x:int(x))
accuracy2 = accuracy_score(df2['test_label'], df2['total_prediction_test'])

In [ ]:
accuracy2

In [ ]:
accuracy = accuracy_score(df['label'], df['total_prediction'])

In [ ]:
accuracy

In [ ]:
accuracy1

In [ ]:
df['total_prediction'].value_counts()

In [ ]:
df1['total_prediction_train'].value_counts()

In [ ]:
df2['total_prediction_test'].value_counts()

In [ ]:
value=df2['test_value'][0]